In [3]:
import pandas as pd
import mysql.connector

# List of CSV file paths["bus_det_final_csvfile_2_kaac.csv"]
csv_files = ["bus_det_final_csvfile_2_kaac.csv","bus_det_final_csvfile_3_meghalaya.csv","bus_det_final_csvfile_5_assam_astc.csv","bus_det_final_csvfile_1_hrtc.csv"]

# Read each CSV file into a DataFrame and concatenate them
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

# Clean data: remove 'INR ' from Price and extract seat numbers from strings
combined_df['Price'] = combined_df['Price'].str.replace('INR ', '')
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract(r'(\d+)')

# Convert Price to decimal and Seat_Availability to integer
combined_df['Price'] = pd.to_numeric(combined_df['Price'], errors='coerce').fillna(0).astype(float)
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].astype(int)

# Convert departing time and reaching time into time format, replacing NaT with None
# Convert departing time and reaching time into time format, replacing invalid times with None
#combined_df['Departing_Time'] = pd.to_datetime(combined_df['Departing_Time'], format='%H:%M:%S', errors='coerce').dt.time
#combined_df['Reaching_Time'] = pd.to_datetime(combined_df['Reaching_Time'], format='%H:%M:%S', errors='coerce').dt.time


# Add auto-increment id column
combined_df.insert(0, 'id', combined_df.index + 1)

# MySQL connection
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="Red_bus_project"
)

mycursor = mydb.cursor()






# Create table query
create_table_query = mycursor.execute("""CREATE TABLE bus_routes3(
    id INT AUTO_INCREMENT PRIMARY KEY,
    route_name TEXT NOT NULL,
    route_link TEXT,
    busname TEXT NOT NULL,
    bustype TEXT NOT NULL,
    departing_time TIME NOT NULL,
    duration TEXT NOT NULL,
    reaching_time TIME NOT NULL,
    star_rating FLOAT,
    price DECIMAL(10, 2) NOT NULL,
    seats_available INT NOT NULL)
    """)

mycursor.execute(create_table_query)

# Insert query (ensure that the column names match exactly with the DataFrame)
insert_query = """
INSERT INTO bus_routes3 (id, route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insert each row from the DataFrame, ensuring None is passed for missing times
for i, row in combined_df.iterrows():
    mycursor.execute(insert_query, tuple(row))

# Commit the changes
mydb.commit()

print("Data inserted successfully.")


Data inserted successfully.


In [ ]:
mycursor.execute("SELECT count(*) FROM ")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))
